## Neo4j Driver

In [1]:
# Import necessary Libraries

import pandas as pd

# !pip3 install neo4j-driver

from neo4j import GraphDatabase, basic_auth

In [17]:
# Connect to neo4j Desktop

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=basic_auth("neo4j", "admin"))
driver.verify_connectivity()

<ipython-input-17-ad7d03d68d2d>:5: ExperimentalWarning: The configuration may change in the future.
  driver.verify_connectivity()


'Neo4j/4.4.5'

In [4]:
# Delete Previous Graph (uncomment if required)

delete_graph = ''' 
MATCH (n)
DETACH DELETE n
'''

with driver.session(database="neo4j") as session:
   session.run(delete_graph)

In [8]:
# Load Distance Matrix CSV File

load_distance = ''' 
LOAD CSV WITH HEADERS FROM 'file:///Mexico_Distance_Matrix.csv' AS row
MERGE (o:City {name: row.Origin})
MERGE (d:City {name: row.Destination})
MERGE (o)-[r:ROAD {distance: toFloat(row.Distance), duration: toFloat(row.Duration)}]->(d)
'''

with driver.session(database="neo4j") as session:
   session.run(load_distance)

In [11]:
# Delete Projected Graph (uncomment if required)

delete_graph = '''
CALL gds.graph.drop('MexicoCrime')
'''

with driver.session(database="neo4j") as session:
   session.run(delete_graph)

In [14]:
# Create a new Graph Projection
project_graph = ''' 
CALL gds.graph.project(
  'MexicoCrime',
  'City',
  {
    ROAD: {
      type: 'ROAD',
      properties: ['distance','duration'],
      orientation: 'UNDIRECTED'
    }
  }
)
'''

with driver.session(database="neo4j") as session:
   session.run(project_graph)

In [15]:
# Run the Minimum Weight Spanning Tree algorithm and write back results to the graph
mst_query = ''' 
MATCH (n:City {name: 'Los Cabos'})
CALL gds.alpha.spanningTree.minimum.write('MexicoCrime', {
  startNodeId: id(n),
  relationshipWeightProperty: 'duration',
  writeProperty: 'MINST',
  weightWriteProperty: 'writeCost'
})
YIELD preProcessingMillis, computeMillis, writeMillis, effectiveNodeCount
RETURN preProcessingMillis, computeMillis, writeMillis, effectiveNodeCount;
'''

with driver.session(database="neo4j") as session:
   session.run(mst_query)

In [27]:
# Query and Print results
cypher_query = '''
MATCH path = (c:City {name: 'Los Cabos'})-[:MINST*]-()
WITH relationships(path) AS rels
UNWIND rels AS rel
WITH DISTINCT rel AS rel
RETURN startNode(rel).name AS origin, endNode(rel).name AS destination, rel.writeCost AS duration
'''

with driver.session(database="neo4j") as session:
  results = session.read_transaction(
    lambda tx: tx.run(cypher_query,
                      limit=10000).data())
  print('Minimum Spanning Tree - Mexico\n')
  for record in results:
    print('From', record['origin'],'to', record['destination'], round(record['duration']//60), 'hours', round(record['duration']%60), 'minutes')

driver.close()

Minimum Spanning Tree - Mexico

From Los Cabos to La Paz 51 hours 44 minutes
From La Paz to Comondú 4 hours 51 minutes
From Comondú to Mulegé 2 hours 21 minutes
From La Paz to Ixtapaluca 49 hours 24 minutes
From Ixtapaluca to Valle de Chalco Solidaridad 0 hours 9 minutes
From Valle de Chalco Solidaridad to Tláhuac 0 hours 12 minutes
From Tláhuac to Milpa Alta 0 hours 15 minutes
From Ixtapaluca to Chalco 0 hours 19 minutes
From Chalco to Amecameca 0 hours 35 minutes
From Chalco to Tlalmanalco 0 hours 19 minutes
From La Paz to Chicoloapan 49 hours 15 minutes
From Chicoloapan to Chimalhuacán 0 hours 9 minutes
From Chimalhuacán to Nezahualcóyotl 0 hours 9 minutes
From Nezahualcóyotl to Iztacalco 0 hours 14 minutes
From Iztacalco to Iztapalapa 0 hours 10 minutes
From Iztacalco to Venustiano Carranza 0 hours 8 minutes
From Venustiano Carranza to Gustavo A. Madero 0 hours 10 minutes
From Venustiano Carranza to Cuauhtémoc 8 hours 7 minutes
From Cuauhtémoc to Azcapotzalco 19 hours 14 minutes
Fr